In [1]:
%reset -fs
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.decomposition import TruncatedSVD, NMF

import re

from corextopic import corextopic as ct

In [2]:
df = pd.read_csv('eu_tweet_df_eda1.csv')

/var/folders/gc/kgfpjt4x1d5c1thdk2sl_2hr0000gn/T/ipykernel_17786/3349357611.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('eu_tweet_df_eda1.csv')


In [3]:
df.head()

,id,created_at,text,user_name,user_screen_name,user_id,user_description,user_location,user_verified,lang,...,in_reply_to_user_id,retweet_count,retweet_id,retweet_screen_name,user_created_at,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
0,866239077045207040,Sun May 21 10:27:44 +0000 2017,Pounding the streets of Redland this morning w...,Green💚🌻Molly,GreenPartyMolly,726372601,Molly Scott Cato 👋 Green Party External Comms ...,South West,False,en,...,NaN,23.0,8.662375e+17,bristol_yg,Mon Jul 30 16:21:34 +0000 2012,27154.0,59652.0,1965.0,854.0,69671.0
1,866232717196840960,Sun May 21 10:02:28 +0000 2017,The UKIP BATTLE BUS is coming; please support ...,Dr Julia Reid PhD,julia_reid,43554444,Former Brexit Party MEP for the SW (2014-19) -...,Calne,True,en,...,NaN,38.0,8.662321e+17,DrTeckKhong,Sat May 30 16:46:20 +0000 2009,62208.0,4773.0,6282.0,242.0,139670.0
2,866236815463862272,Sun May 21 10:18:45 +0000 2017,"Yesterday was 10th European Maritime Day, toda...",Catherine Bearder🇪🇺 🇺🇦🇺🇦🇪🇺🔶,catherinemep,454779231,Lib Dem. Board of IFAW and Council of Unlock D...,Oxford,True,en,...,NaN,21.0,8.662204e+17,EU_MARE,Wed Jan 04 11:13:08 +0000 2012,2705.0,32987.0,9121.0,677.0,36328.0
3,866236868404314112,Sun May 21 10:18:57 +0000 2017,.@theSNP Depute Leader and candidate for Moray...,Ian Hudghton,HudghtonSNP,308677137,SNP President 2005 - 2020. SNP Member of the E...,Scotland,False,en,...,NaN,6.0,8.662227e+17,erikgeddes,Tue May 31 20:22:38 +0000 2011,21.0,5044.0,398.0,205.0,1789.0
4,866233710714257408,Sun May 21 10:06:25 +0000 2017,"@Cacophonic That, and fox hunting and dementia...",Catherine Bearder🇪🇺 🇺🇦🇺🇦🇪🇺🔶,catherinemep,454779231,Lib Dem. Board of IFAW and Council of Unlock D...,Oxford,True,en,...,25067514.0,0.0,NaN,NaN,Wed Jan 04 11:13:08 +0000 2012,2705.0,32987.0,9121.0,677.0,36328.0


In [4]:
def strip_punc_lower_http(txt):
    txt = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", txt)

    txt = re.sub(pattern= "[^\w\s]",
            repl="",
            string=txt).lower()
    
    txt = re.sub(pattern= "[0-9]",
            repl="",
            string=txt)
    
    return txt
strip_punc_lower_http('I am, testing #ALEX > than X 10. https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html')

'i am testing alex  than x   '

In [5]:
df['text'] = df['text'].apply(lambda x: strip_punc_lower_http(x))

In [6]:
df['text'].head()

0    pounding the streets of redland this morning w...
1    the ukip battle bus is coming please support m...
2    yesterday was th european maritime day today i...
3    thesnp depute leader and candidate for moray a...
4    cacophonic that and fox hunting and dementia t...
Name: text, dtype: object

In [7]:
my_stop_words = ['wonderful', 'did', 'didnt', 'issue', 'issues', 'actually', 'id','th',
                'community']

In [8]:
my_stop_words = text.ENGLISH_STOP_WORDS.union(my_stop_words)

In [9]:
tv_180docs = TfidfVectorizer(stop_words=my_stop_words, min_df=180)
dtm_tv_180docs = tv_180docs.fit_transform(list(df['text']))

In [10]:
dtm_tv_180docs.shape

(3189504, 17390)

In [11]:
words_180 = list(np.asarray(tv_180docs.get_feature_names_out()))

In [16]:
corex_model = ct.Corex(n_hidden=  30, ### YOUR NUMBER OF TOPICS HERE
                       words=   words_180,   ### YOUR VOCABULARY HERE
                       seed=1
                       )

corex_model.fit(dtm_tv_180docs,       ### YOUR DOCUMENT TERM MATRIX HERE
                words= words_180   , ### YOUR VOCABULARY HERE
                docs= df['text']     ### YOUR DOCUMENT SERIES HERE
                )

In [17]:
topics = corex_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print(f'{n}: {", ".join(topic_words)}')

0: great, eu, thanks, thank, congratulations, brexit, happy, deal, tax, trump
1: emmyzen, aurorablogspot, julienfouchet, abcpoppins, ulrikeguerot, wernert_man, katarinabarley, inlimbobrexit, w_a_t_o_n, csibike
2: yellabelly, pilgrim, jamieb, bostonpoliceuk, stevemelady, hastieneill, bikerrebel, bostonboro, un_women, mali
3: ist, es, sverige, lgbt, matthancock, mindful, missions, muslim, nah, nearing
4: trolleywatch, inmo, trolleys, wards, monster, ok, peaceful, pmb, precautions, resounding
5: advise, labourdefra, lens, math, michellerempel, nationalhousingstrategy, pfew_hq, portions, qualities, rest
6: attracted, midwives, nchildersmep, pale, personalised, polcan, preserved, railroad, resume, max
7: abundance, newzealand, note, oops, parcel, paulblomfieldmp, pigs, pt, rbkc, repcheri
8: aapi, mmiwg, nzs, otoole, paulmurphy_td, pcs, peace, pharmacies, predecessor, prep
9: activity, playing, preschool, registrations, repsamgraves, richardmarlesmp, scotch, sellers, senatorfischer, shelby
1

In [20]:
corex_model.tcs

array([ 3.96743008e-02,  4.46996306e-04,  2.17814989e-06,  1.09870887e-09,
        2.94944689e-10, -2.42092862e-09, -8.20481412e-09, -1.35065520e-08,
       -1.50525530e-08, -4.20803892e-08])

In [74]:
# words_30_semi = list(np.asarray(tv_30docs.get_feature_names_out()))

In [12]:
anchor_words =[
                ['immigration', 'immigrant'],
                ['christian', 'hindu', 'muslim', 'jewish', 'religion'],
                ['trump'],
                ['brexit', 'eu', 'leave'],
                ['women','violence', 'rights'],
                ['health', 'care','insurance', 'mental'],
                ['climate', 'change'],
                ['jobs', 'economy', 'business']
                ]




corex_semi = ct.Corex(n_hidden= 8, ### YOUR NUMBER OF TOPICS HERE
                       words=   words_180,   ### YOUR VOCABULARY HERE
                       seed=1
                       )

corex_semi.fit(dtm_tv_180docs,       ### YOUR DOCUMENT TERM MATRIX HERE
                words= words_180   , ### YOUR VOCABULARY HERE
                docs= df['text'],
                anchors=anchor_words,
                anchor_strength = 3
                )

In [13]:
topics_semi = corex_semi.get_topics()
for n,topic in enumerate(topics_semi):
    topic_words,_,_ = zip(*topic)
    print(f'{n}: {", ".join(topic_words)}')

0: ist, es, accounts, importantly, inherited, linking, longstanding, magnitude, mould, mystery
1: jewish, muslim, christian, religion, hindu, ramadan, sameach, mubarak, chag, eid
2: academic, paramount, posts, pq, printed, privatising, productivity, professors, reaffirming, redress
3: brexit, eu, leave, deal, uk, referendum, customs, theresa, mays, vote
4: suu, kyi, aung, accountability, rightfully, relaunch, relationships, registering, reg, reconnect
5: health, care, mental, insurance, preexisting, coverage, aca, affordable, protectourcare, premiums
6: climate, change, climatechange, emissions, action, actonclimate, fossil, cop, carbon, climateaction
7: abused, listen, measure, midwife, modest, panorama, parcel, piling, pleas, praises


In [31]:
from corextopic import vis_topic as vt
vt.vis_rep(corex_semi, column_label=words, prefix='topic-model-example')

Print topics in text file


In [15]:
(corex_semi.tcs
 
)

array([ 6.23970167e-06,  4.07306384e-04,  6.86448580e-08,  3.62780707e-02,
       -6.35507287e-08,  1.67863616e-02,  2.81511568e-03,  1.99601536e-08])